In [16]:
import random

import gym
import numpy as np
import tensorflow as tf

In [17]:
env = gym.make("CartPole-v0")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [18]:
num_input_neurons = len(env.reset())
num_ouptut_neurons = env.action_space.n
common_net_hidden_dimensions = [16, 64]

learning_rate = 0.0001

gamma = 0.95

num_episodes = 700
buffer_size = 10000
mini_batch_size = 64
steps_per_target_update = 7
consecutive_successful_episodes_to_stop = 20
min_average_reward_for_stopping = 195

In [19]:
class DDDQN:
    def __init__(self,
                 session,
                 scope_name,
                 input_size,
                 hidden_layer_sizes,
                 output_size,
                 learning_rate):
        
        self.session = session
        self.scope_name = scope_name
        self.input_size = input_size
        self.hidden_layer_sizes = hidden_layer_sizes
        self.output_size = output_size
        self.learning_rate = learning_rate
        
        with tf.variable_scope(self.scope_name):
            self.input = tf.placeholder(shape=[None, self.input_size],
                                        dtype=tf.float32)
            # Logic for common hidden network
            common_net = self.input
            for layer_size in self.hidden_layer_sizes:
                common_net = tf.layers.dense(common_net,
                                             layer_size,
                                             activation=tf.nn.relu)
            
            # Separating streams into advantage and value networks
            adv_net = tf.layers.dense(common_net, 32, activation=tf.nn.relu)
            adv_net = tf.layers.dense(adv_net, self.output_size)
            
            val_net = tf.layers.dense(common_net, 32, activation=tf.nn.relu)
            val_net = tf.layers.dense(val_net, 1)
            
            self.output = val_net + (adv_net - tf.reduce_mean(adv_net,
                                                              reduction_indices=1,
                                                              keepdims=True))

            # Placeholder for expected q-values
            self.y = tf.placeholder(shape=[None, self.output_size], dtype=tf.float32)

            # Using the loss method provided by tf directly
            self.loss = tf.losses.mean_squared_error(self.y, self.output)

            self.optimizer = tf.train.AdamOptimizer(
                learning_rate=self.learning_rate).minimize(self.loss)
    
    def predict(self, state):
        return self.session.run(self.output,
                                feed_dict={self.input: np.reshape(state, [-1, self.input_size])})
    
    def update(self, state, y):
        return self.session.run([self.loss, self.optimizer],
                              feed_dict={
                                    self.input: state,
                                    self.y: y
                              })
    
    @staticmethod
    def create_copy_operations(source_scope, dest_scope):
        source_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=source_scope)
        dest_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=dest_scope)
        
        assert len(source_vars) == len(dest_vars)
        result = []
        
        for source_var, dest_var in zip(source_vars, dest_vars):
            result.append(dest_var.assign(source_var.value()))
        
        return result

In [20]:
def train_dqn(main_dqn, target_dqn, mini_batch):
    """
    param: mini_batch: From the randomly sampled minbatch from replay-buffer,
                       it's a list of experiences in the form of
                       `(state, action, reward, next_state, done)`
    """
    states = [x[0] for x in mini_batch]
    states = np.vstack(states)
    
    actions = np.array([x[1] for x in mini_batch])
    rewards = np.array([x[2] for x in mini_batch])
    next_states = np.vstack([x[3] for x in mini_batch])
    done = np.array([x[4] for x in mini_batch])
    
    target_output_next_states = target_dqn.predict(next_states)
    # For double DQN: select the best action for next state
    main_output_next_states = main_dqn.predict(next_states)
    
    selected_best_actions = np.argmax(main_output_next_states, axis=1)
    target_output_for_selected_actions = target_output_next_states[
        np.arange(len(states)), selected_best_actions]
    
#     target_q_vals = rewards + gamma * np.max(target_output, axis=1) * (1 - done)

    target_q_vals = rewards + gamma * target_output_for_selected_actions * (1 - done)
    
    main_output = main_dqn.predict(states)
    main_output[np.arange(len(states)), actions] = target_q_vals
    
    loss, optimizer = main_dqn.update(states, main_output)
    
    return loss

In [24]:
from collections import deque
tf.reset_default_graph()

replay_buffer = deque(maxlen=buffer_size)
last_n_rewards = deque(maxlen=consecutive_successful_episodes_to_stop)

with tf.Session() as sess:
    main_dqn = DDDQN(session=sess,
                   scope_name="q_main",
                   input_size=num_input_neurons,
                   hidden_layer_sizes=common_net_hidden_dimensions,
                   output_size=num_ouptut_neurons,
                   learning_rate=learning_rate)
    
    target_dqn = DDDQN(session=sess,
                     scope_name="q_target",
                     input_size=num_input_neurons,
                     hidden_layer_sizes=common_net_hidden_dimensions,
                     output_size=num_ouptut_neurons,
                     learning_rate=learning_rate)
    
    sess.run(tf.global_variables_initializer())

    # Make them identical to begin with
    sess.run(DDDQN.create_copy_operations("q_main", "q_target"))
    
    for ep_num in range(num_episodes):
               
        state = env.reset()
        done = False
        episode_reward, loss, steps = 0, 0, 0

        # epsilon decay
        epsilon = 1. /((ep_num / 10) + 1)
        
        while not done:
            # select the action
            action = None
            if np.random.rand() < epsilon:
                action = env.action_space.sample()
            else:
                action = np.argmax(main_dqn.predict(state))
            
            # execute the action
            next_state, reward, done, _ = env.step(action)
            
            if done:
                reward = -1
            
            # add to the buffer
            replay_buffer.append((state, action, reward, next_state, done))
            
            # sample from the buffer and train
            if len(replay_buffer) > mini_batch_size:
                mini_batch = random.sample(replay_buffer, (mini_batch_size))
                loss = train_dqn(main_dqn, target_dqn, mini_batch)
            
            if steps % steps_per_target_update == 0:
                sess.run(DDDQN.create_copy_operations("q_main", "q_target"))    
            
            
            episode_reward += reward
            steps += 1
            state = next_state
            

        last_n_rewards.append(episode_reward)
        last_n_avg_reward = np.mean(last_n_rewards)
        
        # Stopping criteria
        if len(last_n_rewards) == consecutive_successful_episodes_to_stop \
            and last_n_avg_reward > min_average_reward_for_stopping:
                print("Solved after {} epsiodes".format(ep_i))
                break

Solved after 166 episodes
